# Batch Daily Inference Pipeline
* Retrieve best model from Hopsworks
* Scrape weather forecasts for next 14 days
* Predict weather code and store predictions in Hopsworks

In [1]:
import hopsworks
import joblib
from weather_utils import *
from config import *

# 1) Retrieve best model from Hopsworks

In [2]:
# Connect to Hopsworks
project = hopsworks.login()

# Retrieve Feature Store
fs = project.get_feature_store()

# Get Model Registry
mr = project.get_model_registry()

# Select best model based on evaluation metric
weather_code_model = mr.get_best_model(MODEL_NAME,
                          MODEL_METRIC,
                          OPTIMIZE_DIRECTION)

# Download model path
model_dir = weather_code_model.download()

# Download model
model = joblib.load(model_dir + '/'+ MODEL_NAME + '.pkl')

print("Model:", model_dir)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/178324
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


# 2) Scrape weather forecasts for next 14 days

In [3]:
# Define query parameters
params = {
    "latitude": LATITUDE,
    "longitude": LONGITUDE,
    "daily": ["temperature_2m_min", "precipitation_sum", "wind_gusts_10m_max"],
    "timezone": TIMEZONE,
    "past_days": 0,
    "forecast_days": 14
}

# Setup connection with Open-Meteo
openmeteo = get_openmeteo_connection()

# Execute the query
responses = openmeteo.weather_api(BASELINE_URL_OPEN_METEO, params=params)

# Extract the location because the response can be done for multiple cities as well
response = responses[0]

# Process yesterday's data into a Pandas dataframe
df_forecasts = process_forecast_request(response)

print(df_forecasts)

          date  temperature_min  precipitation_sum  wind_gusts_max
0   2023-11-13        10.287001           3.300000       59.759998
1   2023-11-14         9.037001           1.400000       41.399998
2   2023-11-15         5.661000           0.100000       28.440001
3   2023-11-16         4.261000           0.000000       19.440001
4   2023-11-17         5.861000           6.899999       42.119999
5   2023-11-18         9.861000           6.399999       54.000000
6   2023-11-19         6.335000           6.499999       43.919998
7   2023-11-20         3.135000           0.300000       38.880001
8   2023-11-21         6.161000           0.300000       36.000000
9   2023-11-22         7.111000           0.900000       45.360001
10  2023-11-23         4.111000           0.000000       48.599998
11  2023-11-24         2.861000           3.000000       46.439999
12  2023-11-25         1.061000           0.000000       16.919998
13  2023-11-26         2.511000           0.900000       39.23